In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pandas as pd
df = pd.read_csv('vessel_label_new.csv')
target = df.loc[:,'Class']
df['Class'].value_counts()

In [ ]:
from collections import Counter
fold_no=1
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
skfold = StratifiedKFold(n_splits=9, shuffle=True, random_state=1)
x_tr, x_val, y_tr, y_val = train_test_split(df, target, test_size=0.20, random_state=67,stratify=Y)


for train_index, val_index in skfold.split(df, target):
    model_history = []
    fold = "fold_" + str(fold_no)
    print("FOLD: ", fold )
    print (" - train -")
  # print("FOLD: ", fold_no , "TRAIN:", train_index, "TEST:", val_index, target[val_index])
    train = df.loc[train_index,:]
    print (train['Class'].value_counts())
#     print(train)
    save_train_path='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/train'
    path_train=filter_subjects(fold, "train", train,save_train_path)
  
    
    print ("- valid - ")
    val = df.loc[val_index,:]
    print (val['Class'].value_counts())
    print ("\n")
    save_val_path='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/val_subject'
    path_val=filter_subjects(fold, "valid", val,save_val_path)
    
    print(path_val)
    p_t='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/train'
    path_train=os.path.join(p_t,'fold_'+str(fold_no),'train')
    
    p_v='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/val'
    path_val=os.path.join(p_v,'fold_'+str(fold_no),'valid')
    print(path_train,path_val)
    test=val.loc[:,'value']
    true_lb.append(test)
    
#     print(true_lb)
    Y = train.loc[:,'Class']
    x_tr, x_val, y_tr, y_val = train_test_split(train, Y, test_size=0.20, random_state=67,stratify=Y)
    
# #     print(y_tr)


    save_x_tr='E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/train_validation/train'
    path_x_tr=filter_subjects(fold, "train", x_tr,save_x_tr)
    
    save_x_val='E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/train_validation/val'
    path_x_val=filter_subjects(fold, "val", x_val,save_x_val)
    
    
    p_x_tr='E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/train_validation/train'
    path_x_tr=os.path.join(p_x_tr,'fold_'+str(fold_no),'train')
    
    p_x_val='E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/train_validation/val'
    path_x_val=os.path.join(p_x_val,'fold_'+str(fold_no),'val')


In [ ]:
def filter_subjects(fold, dataset_type, df,path):
    source_folder='E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/train_test'
    destination_folder=path
  # for each of the subjects in the dataframe
    for ind in df.index:
        subject = df['subject_list'][ind]
#         print(subject)
    # for each of the files in a directory
        for folders,_,filenames in os.walk(source_folder):  
      # for each of the filenames 
#           print(subfolders)
          for file in filenames:
#             print(file)
        # if a subject is found, clone it to a folder with its class name. 
            # prefix = file[0:file.rindex("_")]
          
#             if subject.startswith(prefix):   
#             print(prefix)
            if subject.split("_")[1] == file.split("_")[1]:
#               print(subject, prefix)
              
              source = os.path.join(folders, file)
#               print(source)
              destination = os.path.join(destination_folder, fold, dataset_type, df['Class'][ind])
#             print(destination)
              from pathlib import Path
          # check if destination folder exists; if not create.
              Path(destination).mkdir(parents=True, exist_ok=True)
#                 print(source, destination)
              import shutil
              shutil.copy2(source, destination)
    return destination

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip("vertical"),
        layers.RandomContrast(factor=0.5)
#         layers.GaussianNoise(.2, seed=None)
    ],
    name="img_augmentation",
)

In [ ]:
def cnn_model():
    from tensorflow.keras.applications.vgg16 import VGG16
    from keras.layers import Dropout
    vggmodel = VGG16(weights='imagenet', include_top=True)
#     vggmodel = VGG16(weights='imagenet', include_top=True)
    for layers in (vggmodel.layers)[:19]:
#     print(layers)
        layers.trainable = False
   
    from tensorflow import keras
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense
    from tensorflow.keras import optimizers

    from tensorflow.keras.preprocessing import image
    from keras.layers import Input, Flatten, Dense
    from keras.models import Model
    fc1 = vggmodel.layers[-3]
    fc2 = vggmodel.layers[-2]
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)


# Create the dropout layers
    dropout1 = Dropout(0.70)
    dropout2 = Dropout(0.70)
    output_vgg16 = vggmodel.output

#Add the fully-connected layers 
#     x = Flatten(name='flatten')(output_vgg16)
#     x = Dense(2048, activation='relu', name='fc1')(x)
#     x = dropout1(x)
#     x = Dense(1024, activation='relu', name='fc2')(x)
#     x = dropout2(x)
    
#     x = Dense(3, activation='softmax', name='predictions')(x)

    # Reconnect the layers
    x = dropout1(fc1.output)
    x = fc2(x)
    x = dropout2(x)
#     X= vggmodel.layers[-2].output
    predictions = Dense(3, activation="softmax")(x)
    model_final = Model(inputs = vggmodel.input, outputs = predictions)

    epochs=20
    learning_rate = 0.0001
    decay_rate = learning_rate / epochs
    momentum = 0.8
    model_final.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['AUC'])
#     model_final.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule),
#               loss=tf.keras.losses.CategoricalCrossentropy(),
#               metrics=['AUC'])
#     model_final.summary()
    return model_final


In [ ]:
def cnn_model():
    from tensorflow.keras.applications import EfficientNetB0
    inputs = layers.Input(shape=(224, 224, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.7
    x = layers.Dropout(top_dropout_rate, name="top_dropout2")(x) 
    outputs = layers.Dense(2, activation="softmax", name="pred")(x)
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")

    return model

In [ ]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', np.unique(train.classes), train.classes)
cw = dict(zip( np.unique(train.classes), weights))
print(cw)

In [ ]:
def fit_and_evaluate(t_x,t_y, val_x,val_y, i, EPOCHS=100, BATCH_SIZE=16):
    model = None
    model = cnn_model()
    class_weight={0:.7,1:1.89,2:2.194}
#     t_y = to_categorical(t_y)
#     val_y = to_categorical(val_y)
#     print(len(t_x))
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    aug = ImageDataGenerator(
#     rotation_range=30,
#     zoom_range=0.15,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.15,
#     vertical_flip=True,
    horizontal_flip=True)

    results = model.fit(x=aug.flow(t_x,t_y,batch_size=BATCH_SIZE),validation_data=(val_x,val_y),batch_size=BATCH_SIZE,epochs=epochs,class_weight=class_weight,callbacks=[early_stopping, model_checkpoint], verbose=1)  
#     model.load_weights("E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/saved_models/model_"+str(i)+".h5")
#     print("Val Score: ", model.evaluate(val_x, val_y))
    return results

In [ ]:
def fit_and_evaluate1(train_ds,val_ds, i, EPOCHS=20, BATCH_SIZE=32):
    model = None
    model = cnn_model()
    class_weight={0:.7,1:1.5,2:2.7}
#     t_y = to_categorical(t_y)
#     val_y = to_categorical(val_y)
#     print(len(t_x))
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    aug = ImageDataGenerator(
#     rotation_range=30,
#     zoom_range=0.15,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.15,
    vertical_flip=True,
    horizontal_flip=True)
#     fill_mode="nearest")
    
    results = model.fit(x=aug.flow(t_x, t_y,batch_size=BATCH_SIZE),validation_data=(val_x,val_y),epochs=EPOCHS,class_weight=class_weight,callbacks=[early_stopping, model_checkpoint], 
              verbose=1)  
# ,validation_data=(val_x,val_y)
#     results = model.fit(train_ds,validation_data=val_ds,epochs=epochs,class_weight=class_weight,callbacks=[early_stopping, model_checkpoint], verbose=1)  
#     model.load_weights("E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/saved_models/model_"+str(i)+".h5")
#     print("Val Score: ", model.evaluate(val_x, val_y))
    return results

In [ ]:
def data_prepare1(path):   # prepare data augmentation configuration
    from keras.applications.vgg16 import preprocess_input
    from keras.preprocessing.image import ImageDataGenerator
    from dynamic_sampling import DynamicSamplingImageDataGenerator, BatchSizeAdapter
    import random
    import numpy as np


# Prepare data-augmenting data generator
    from keras.preprocessing.image import ImageDataGenerator
    datagen_train = ImageDataGenerator()
    
    
    img_height=224
    img_width=224
    batch_size=16
    train_generator = datagen_train.flow_from_directory(
        path,
#         class_mode='sparse',
        target_size=(img_height, img_width))

    lev=train_generator.classes

    return train_generator,lev


In [ ]:
def data_prepare2(path):   # prepare data augmentation configuration
    from keras.applications.vgg16 import preprocess_input
    from keras.preprocessing.image import ImageDataGenerator

    img_height=224
    img_width=224
    batch_size=16
    test_datagen = ImageDataGenerator()

    test_generator = test_datagen.flow_from_directory(
        path,
#         class_mode='sparse',
        target_size=(img_height, img_width),
        batch_size=batch_size)
    lev=test_generator.classes

    return test_generator,lev


In [ ]:
def data_prepare3(path):   # prepare data augmentation configuration
    from keras.applications.vgg16 import preprocess_input
    from keras.preprocessing.image import ImageDataGenerator

    img_height=224
    img_width=224
    batch_size=1
    test_datagen = ImageDataGenerator()

    test_generator = test_datagen.flow_from_directory(
        path,
#         class_mode='sparse',
        shuffle=False,
        target_size=(img_height, img_width),
        batch_size=batch_size)
    lev=test_generator.classes

    return test_generator,lev

In [ ]:
def test_subject(path_test,model):

    pr=[]

    import glob
    import cv2
    img_height=224
    img_width=224
    for img in glob.glob(os.path.join(path_test, '*.jpg')):
        img = cv2.imread(img, cv2.IMREAD_COLOR)
        dim = (224, 224)
        img=cv2.resize(img, dim,interpolation = cv2.INTER_AREA)
        img_array = tf.expand_dims(img, 0) # Create a batch
        predictions = model.predict(img_array)
        pred=np.argmax(predictions,axis=1)
        pr.append(pred)
    
    predictions= [ item for elem in pr for item in elem]

    print('Flat List : ',predictions)
    print(len(predictions))
    
    from collections import Counter

    def find_majority(votes):
        vote_count = Counter(votes)
        print(vote_count)
        top_two = vote_count.most_common(2)
        if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
            return 0
        return top_two[0][0]

    result=find_majority(predictions)
    return result

In [ ]:
def focal_loss_custom(alpha, gamma):
    def binary_focal_loss(y_true, y_pred):
        import tensorflow_addons as tfa
        fl = tfa.losses.SigmoidFocalCrossEntropy(alpha=alpha, gamma=gamma)
        y_true_K = K.ones_like(y_true)
        focal_loss = fl(y_true, y_pred)
        return focal_loss
    return binary_focal_loss

In [ ]:
def focal_tverskyLoss(y_true, y_pred, smooth=1e-6):
    
        if y_pred.shape[-1] <= 1:
            alpha = 0.3
            beta = 0.7
            gamma = 4/3 #5.
            y_pred = tf.keras.activations.sigmoid(y_pred)
            #y_true = y_true[:,:,:,0:1]
        elif y_pred.shape[-1] >= 2:
            alpha = 0.3
            beta = 0.7
            gamma = 4/3 #3.
            y_pred = tf.keras.activations.softmax(y_pred, axis=-1)
            y_true = K.squeeze(y_true, 3)
            y_true = tf.cast(y_true, "int32")
            y_true = tf.one_hot(y_true, num_class, axis=-1)
           
        y_true = K.cast(y_true, 'float32')
        y_pred = K.cast(y_pred, 'float32')
        #flatten label and prediction tensors
        inputs = K.flatten(y_pred)
        targets = K.flatten(y_true)
        
        #True Positives, False Positives & False Negatives
        TP = K.sum((inputs * targets))
        FP = K.sum(((1-targets) * inputs))
        FN = K.sum((targets * (1-inputs)))
       
        Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
        
        return 1 - Tversky

In [ ]:
def check_individual(model,fold_no): 
    import cv2
    from tensorflow.keras.models import load_model
    p_v='val_2'
    save_dir =  'saved_models/'
#     fold_no=1
    all_subjects_g=[]
    all_subjects_i=[]
    all_subjects_p=[]
#     for i in range(1,10):
    path_val_g=os.path.join(p_v,'fold_'+str(fold_no),'valid','good')
#     path_val_i=os.path.join(p_v,'fold_'+str(fold_no),'valid','intermediate')
    path_val_p=os.path.join(p_v,'fold_'+str(fold_no),'valid','poor')
    print(path_val_g,path_val_p)

    predicted_label_g=test_subject(path_val_g,model)
#     predicted_label_i=test_subject(path_val_i,model)
    predicted_label_p=test_subject(path_val_p,model)
    
#     print("predicted",predicted_label)
    all_subjects_g.append(predicted_label_g)
#     all_subjects_i.append(predicted_label_i)
    all_subjects_p.append(predicted_label_p)
    print('fold:',fold_no, all_subjects_g, all_subjects_p)
        
#         fold_no= fold_no + 1
    return None
    

In [ ]:
# from network import VGG16
# from keras.preprocessing import image
# from keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from keras.models import Model
# from keras import optimizers 
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import StratifiedShuffleSplit
import cv2


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from keras.layers import Dropout
from keras.layers import Dense, GlobalAveragePooling2D
   
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing import image
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import backend as K
from tensorflow.keras.utils import to_categorical

import time

tic = time.perf_counter()
    
p_x_tr='train'


p_x_val='val'
callbacks=[]
weights={0:1, 1:4,2:1}
fold_no=1
for i in range(1,10):
    path_x_tr=os.path.join(p_x_tr,'fold_'+str(fold_no),'train')
    path_x_val=os.path.join(p_x_val,'fold_'+str(fold_no),'val')  
    
    t_x,t_y=data_prepare1(path_x_tr)
    val_x,val_y=data_prepare2(path_x_val)
    
    trainX=t_x
   
    testX=val_x
    testy=val_y
    
    datagen_train=trainX
    datagen_valid=testX
    


    save_dir =  'saved_models/focal_loss_weight/'
    checkpoint_path=save_dir+get_model_name(fold_no)
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    from tensorflow.keras.models import load_model
    

    model=None
    model=cnn_model()
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 7, factor= 0.5, min_lr= 1e-7, verbose=1)
    ep=200
    
    from collections import Counter
    c=Counter(t_y)
    total=c[0]+c[1]
    weight_for_0 = (1 / c[0]) * (total / 2.0)
    weight_for_1 = (1 / c[1]) * (total / 2.0)

    weights = {0: weight_for_0, 1: weight_for_1}

    print('Weight for class 0: {:.2f}'.format(weight_for_0))
    print('Weight for class 1: {:.2f}'.format(weight_for_1))
    
    # Create hyperparameter space
    epochs = [2, 4]
    batches = [8, 16, 32, 64]
    lr=[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]
    optimizers = [tf.keras.optimizers.rmsprop(learning_rate=lr), tf.keras.optimizers.Adam(learning_rate=lr)]
    loss_f=[focal_loss_custom(alpha=0.2, gamma=2.0),focal_tverskyLoss]

    from focal_loss import BinaryFocalLoss
#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizers, loss=loss_f, metrics=["accuracy"]
    )

    model_history = model.fit(datagen_train, validation_data =datagen_valid,batch_size=batches,callbacks=[rlrop,early_stop],class_weight=weights,verbose = 1)
#saving the trained model weights as data file in .h5 format
    filename=save_dir+get_model_name(fold_no)
    # View hyperparameters of best neural network
    #grid_result.best_params_
   
    print("Total parameters searched:")
    print("   ",hyperparameters)
    print("Best parameters are:")
    print("   ",model_history.best_params_)
    print("With a score of:")
    print("   ",model_history.best_score_)
    model.save_weights(filename)
    
    ##########--------------new case testing--------------######
    p_t='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/val_2'
    path_test=os.path.join(p_t,'fold_'+str(fold_no),'valid')
    newX,newy=data_prepare3(path_test)
#     loss,accuracy = model.evaluate(newX)
#     print('Test accuracy :', accuracy)
    
    from sklearn.metrics import classification_report

    predIdxs = model.predict(newX)
    predIdxs = np.argmax(predIdxs, axis=1)
    print(classification_report(newy, predIdxs))
    
    check_individual(model,fold_no)
#     filename=save_dir+get_model_name(fold_no)
#     model.save(filename)

    toc = time.perf_counter()
    print(f"one fold is taking {toc - tic:0.4f} seconds to complete")

    fold_no= fold_no + 1

In [ ]:
import matplotlib.pyplot as plt
hist1=model_history

def plot_hist(hist):
    plt.plot(hist.history["loss"])
#     plt.plot(hist.history["val_loss"])
#     plt.plot(hist.history["lr"])
    plt.title("model loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(hist1)

In [ ]:
import cv2
from tensorflow.keras.models import load_model
p_v='C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/val'
save_dir =  'E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/saved_models/'
fold_no=1
all_subjects_g=[]
all_subjects_i=[]
all_subjects_p=[]
for i in range(9):
    path_val_g=os.path.join(p_v,'fold_'+str(fold_no),'valid','good')
    path_val_i=os.path.join(p_v,'fold_'+str(fold_no),'valid','intermediate')
    path_val_p=os.path.join(p_v,'fold_'+str(fold_no),'valid','poor')
    print(path_val_g,path_val_i,path_val_p)
        
    model=None
    model=cnn_model()
    model.load_weights(save_dir+get_model_name(fold_no))
#     model=load_model(save_dir+get_model_name(fold_no)) 
    predicted_label_g=test_subject(path_val_g,model)
    predicted_label_i=test_subject(path_val_i,model)
    predicted_label_p=test_subject(path_val_p,model)
    
#     print("predicted",predicted_label)
    all_subjects_g.append(predicted_label_g)
    all_subjects_i.append(predicted_label_i)
    all_subjects_p.append(predicted_label_p)
    print('fold:',i+1, all_subjects_g, all_subjects_i,all_subjects_p)
    fold_no= fold_no + 1
    
    
    

In [ ]:
data_dir_test='E:/mumu/data_2d_from_4d_mip_filtered/test/poor'
# class_names=test_dataset.class_names
pr=[]
#model=load_model('testmodel_gc.h5')
model=cnn_model()
model.load_weights('E:/mumu/data_2d_from_4d_mip_filtered/all_corrected/stacked_overlap_classification/saved_models/model_9.h5')
import glob
import cv2
img_height=221
img_width=221
for img in glob.glob("C:/Users/m_ktar/Downloads/VGG16-In-Keras-master/val/fold_9/valid/intermediate/s_84_*.jpg"):
    img = cv2.imread(img, cv2.IMREAD_COLOR)
    dim = (224, 224)
    img=cv2.resize(img, dim,interpolation = cv2.INTER_AREA)
    img_array = tf.expand_dims(img, 0) # Create a batch
#     print(img_array.shape)
    predictions = model.predict(img_array)

    pred=np.argmax(predictions,axis=1)
#     print(pred)
    pr.append(pred)
    
pred= [ item for elem in pr for item in elem]

print('Flat List : ',pred)
print(len(pred))

from collections import Counter

def find_majority(votes):
    vote_count = Counter(votes)
    print(vote_count)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
        return 0
    return top_two[0][0]

result=find_majority(pred)
print(result)